In [ ]:
import os

if os.name == 'nt':
    measuring_root = "D:/measuring"
else:
    measuring_root = "/Users/jesse/Thesis/Code"
    
execfile(os.path.join(measuring_root, "analysis/scripts/setup_analysis.py"))
import analysis.lib.purification.purify_delayfb as pu_delayfb
from analysis.lib.tools import toolbox
reload(pu_delayfb)
%matplotlib inline

In [ ]:
ts_start = "20170801232700"
ts_end = "20170803130500"

f_template = "LDE_phase_calibration_C%d"

carbons = [2,4,5,3,6,7]
folders = {}
for c in carbons:
    folders[str(c)] = toolbox.latest_data(contains=f_template % c, newer_than=ts_start, older_than=ts_end, return_all=True)[::-1]

In [ ]:
reload(pu_delayfb)

data = {}

for c in carbons:
    data[str(c)] = dict()
    for f in folders[str(c)]:
        dt_str = "".join(toolbox.get_date_time_string_from_folder(f))
        data[str(c)][dt_str] = pu_delayfb.calibrate_LDE_phase(
            folder=f, 
            older_than=ts_end,
            fixed=[1],
            show_guess=True,
            phi0=180.0,
            ret=True,
            show_plot=True,
            do_fit=True
        )

In [ ]:
time_points = len(folders['2'])

phase_dev = np.zeros((len(carbons), time_points))
phase_dev_u = np.zeros_like(phase_dev)
times = np.zeros_like(phase_dev)

first_ts = toolbox.get_datetime_from_folder(folders['2'][0])

for i_c, c in enumerate(carbons):
    c_key = str(c)
    for i_dt, dt in enumerate(data[c_key].iterkeys()):
        phase_dev[i_c,i_dt] = data[c_key][dt]['acq_phase_per_rep']
        phase_dev_u[i_c,i_dt] = data[c_key][dt]['u_acq_phase_per_rep']
        times[i_c,i_dt] = (toolbox.datetime_from_timestamp(dt) - first_ts).total_seconds()

In [ ]:
import measurement.scripts.lt4_scripts.setup.msmt_params as msmt_params
reload(msmt_params)

trans = "m1"

couplings = []
deviations = []

for c in carbons:
    print ("Carbon: %d" % c)
    ms0_freq = msmt_params.cfg['samples']['111no2']['C%d_freq_0' % c]
    ms1_freq = msmt_params.cfg['samples']['111no2']['C%d_freq_1_%s' % (c, trans)]
    avg_freq = msmt_params.cfg['samples']['111no2']['C%d_freq_%s' % (c, trans)]
    
    parallel_coupling = ms1_freq - ms0_freq
    print("parallel coupling (kHz): \t%.3f" % (parallel_coupling*1e-3))
    
    couplings.append(parallel_coupling)
    
couplings = np.array(couplings) * 1e-3

In [ ]:
plt.figure(figsize=(10,10))

i_offset = 0
for i_c in np.argsort(couplings):
    c = carbons[i_c]
    plt.errorbar(times[i_c,:]/3600, phase_dev[i_c,:]+i_offset, phase_dev_u[i_c,:], fmt='o', label="C%d (%d kHz)" % (c, couplings[i_c]))
    i_offset += 1

plt.legend(loc=2, bbox_to_anchor=(1,1))
plt.xlabel("time (h)")
plt.ylabel("LDE phase deviation (degrees)")

In [ ]:
np.argsort(couplings)